<a href="https://colab.research.google.com/github/swati-git/FineTuneLLM/blob/main/FineTuning_a_LLM_LIMA_CPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers==4.57.3  peft==0.5.0 trl==0.19.1

In [2]:
!pip install -q torch==2.9.0

In [3]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained("facebook/opt-1.3b")
print(config.dtype)

torch.float16


In [4]:
print(f"Model name: {config.model_type}")
print(f"Hidden size: {config.hidden_size}")           # 2048
print(f"Number of layers: {config.num_hidden_layers}") # 24
print(f"Vocabulary size: {config.vocab_size}")        # 50272
print(f"Max sequence length: {config.max_position_embeddings}") # 2048

Model name: opt
Hidden size: 2048
Number of layers: 24
Vocabulary size: 50272
Max sequence length: 2048


In [5]:
from transformers import AutoModelForCausalLM
import torch


model = AutoModelForCausalLM.from_pretrained("facebook/opt-1.3b", torch_dtype=torch.bfloat16)

`torch_dtype` is deprecated! Use `dtype` instead!


In [6]:
def get_model_size(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    # Calculate memory (in GB)
    bytes_per_param = 2 if str(model.dtype) == "torch.bfloat16" else 4
    memory_gb = (total_params * bytes_per_param) / (1024**3)

    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    print(f"Model size in memory: {memory_gb:.2f} GB")
    print(f"Data type: {model.dtype}")



In [7]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [8]:
print_trainable_parameters(model)

trainable params: 1315758080 || all params: 1315758080 || trainable%: 100.0


In [9]:
get_model_size(model)

Total parameters: 1,315,758,080
Trainable parameters: 1,315,758,080
Model size in memory: 2.45 GB
Data type: torch.bfloat16


In [10]:
#Rule of thumb: Need 3-4x model size for training (gradients, optimizer states, etc.)
#2.6 GB model → need ~8-10 GB GPU for training

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

# ===== CHECK THESE =====
print(f"Vocab size (tokenizer): {len(tokenizer)}")
print(f"Vocab size (model): {model.config.vocab_size}")

# These should match!
#assert len(tokenizer) == model.config.vocab_size, "Mismatch!"

# Check special tokens
print(f"Padding token: {tokenizer.pad_token}")
print(f"EOS token: {tokenizer.eos_token}")
print(f"BOS token: {tokenizer.bos_token}")

# Test tokenization
sample = "Write a product description for headphones"
tokens = tokenizer.encode(sample)
print(f"Sample tokenization: {tokens}")
print(f"Number of tokens: {len(tokens)}")

Vocab size (tokenizer): 50265
Vocab size (model): 50272
Padding token: <pad>
EOS token: </s>
BOS token: </s>
Sample tokenization: [2, 45714, 10, 1152, 8194, 13, 15684]
Number of tokens: 7


In [12]:
!pip install -q deeplake==3.7.1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.0.0 requires dill<0.3.9,>=0.3.0, but you have dill 0.4.0 which is incompatible.
datasets 4.0.0 requires multiprocess<0.70.17, but you have multiprocess 0.70.18 which is incompatible.


In [13]:
import deeplake

# Connect to the training and testing datasets
ds = deeplake.load('hub://genai360/GAIR-lima-train-set')
ds_test = deeplake.load('hub://genai360/GAIR-lima-test-set')

/usr/local/lib/python3.12/dist-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (4.4.4) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(
\

Opening dataset in read-only mode as you don't have write permissions.


/usr/local/lib/python3.12/dist-packages/deeplake/core/fast_forwarding.py:43: UserWarning: Loading a dataset that was created or updated with a newer version of deeplake. This could lead to corruption or unexpected errors! Dataset version: 3.7.2, current deeplake version: 3.7.1. It's recommended that you update to a version of deeplake >= 3.7.2.
  warnings.warn(

|

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/genai360/GAIR-lima-train-set



/

...

Opening dataset in read-only mode as you don't have write permissions.


/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/genai360/GAIR-lima-test-set



-

hub://genai360/GAIR-lima-test-set loaded successfully.



In [14]:
ds

Dataset(path='hub://genai360/GAIR-lima-train-set', read_only=True, tensors=['answer', 'embeddings', 'question', 'source'])

In [15]:
#!pip install pipdeptree

In [16]:
#!pipdeptree -p transformers

In [17]:
def prepare_sample_text(example):
    """Prepare the text from a sample of the dataset."""
    text = f"Question: {example['question'].text()}\n\nAnswer: {example['answer'].text()}"
    return text

In [18]:
#!pip install -q trl==0.26.2
#https://github.com/unslothai/unsloth/issues/3057


In [19]:
#Given that the model's max sequence length is 2048 tokens as per  "{config.max_position_embeddings}" we'll structure our dataset to match it.

from trl.trainer import ConstantLengthDataset

train_dataset = ConstantLengthDataset(
    tokenizer,
    ds,
    formatting_func=prepare_sample_text,
    infinite=True,
    seq_length=2048
)



In [20]:
train_dataset

In [26]:
from trl.trainer import ConstantLengthDataset

eval_dataset = ConstantLengthDataset(
    tokenizer,
    ds_test,
    formatting_func=prepare_sample_text,
    infinite=True,
    seq_length=2048
)


**Rank Selection Guidelines**

Small models (< 1B parameters): 8-16

Medium models (1B-10B): 16-32

Large models (> 10B): 32-64

**Alpha-to-Rank Relationship**

Typically set to r or 2 * r

Higher alpha increases the adaptation's impact

Lower alpha reduces the adaptation's influence

In [22]:


from peft import LoraConfig

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

In [25]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./OPT-fine_tuned-LIMA-CPU",
    dataloader_drop_last=True,
    #evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,
    logging_steps=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    warmup_steps=100,
    gradient_accumulation_steps=1,
    #bf16=True,
    weight_decay=0.05,
    run_name="OPT-fine_tuned-LIMA-CPU",
    report_to="wandb",
)

In [28]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=lora_config,
    #packing=True,
)

In [ ]:
print("Training...")
trainer.train()